In [1]:
import json
import os

In [2]:
def lines_to_text(dialogue_lines):
    return "\n".join([d["character"] + ": " + d["dialogue"] for d in dialogue_lines])

def create_prompts(annotation_dir, n_lines):
    with open(annotation_dir, 'r') as file:
        jdata = json.load(file)

    character_info = jdata[0]['All Characters']
    dialoguelist = jdata[1:]

    dataset = []
    n_lines = 1

    for section in dialoguelist:
        characters = section["characters"]

        character_descriptions = ""
        for char in characters:
            if char in character_info:
                character_descriptions += char + ": " + character_info[char] + '\n'
        
        dialogue_history = []
        dialogue_lines = section["dialogue"]

        for i in range(1, len(dialogue_lines)-n_lines):
            history = lines_to_text(dialogue_lines[max(0,i-10):i])

            prompt_text = ("[INST] Character Info: " + character_descriptions + "\n\n"
                        + "Dialogue History: " + history + '\n\n'
                        + "Instruction: Continue the dialogue for " + str(n_lines) + " lines. \n [/INST]")
            ground_truth = lines_to_text(dialogue_lines[i:i+n_lines])
            
            dataset.append({"prompt": prompt_text, "ground_truth": ground_truth})
    return dataset
# for file in os.listdir('./'):



In [11]:
n_lines = 3
full_dataset = []
for filename in os.listdir('./data/'):
    if 'annotated' not in filename:
        continue
    full_path = './data/' + filename
    output_path = full_path.split('conversations')
    output_path[1] = output_path[1].split('_annotated')[0]
    output_path = output_path[0] + f'partial_prompts/prompts_n{n_lines}' + output_path[1] + '.json'
    dataset = create_prompts(full_path, n_lines)
    with open(output_path, 'w') as f:
        json.dump(dataset, f, indent=4)
    full_dataset += dataset
with open(f'./data/prompts_full_n{n_lines}.json', 'w') as f:
    json.dump(full_dataset, f, indent=4)

In [3]:
import random
n_lines = 1
full_dataset = []
for filename in os.listdir('./data/'):
    if 'annotated' not in filename:
        continue
    full_path = './data/' + filename
    output_path = full_path.split('conversations')
    output_path[1] = output_path[1].split('_annotated')[0]
    output_path = output_path[0] + f'partial_prompts/prompts_n{n_lines}' + output_path[1] + '.json'
    dataset = random.sample(create_prompts(full_path, n_lines),10)
    with open(output_path, 'w') as f:
        json.dump(dataset, f, indent=4)
    full_dataset += dataset
with open(f'./data/prompts_sample_n{n_lines}.json', 'w') as f:
    json.dump(full_dataset, f, indent=4)